In [1]:
print("okie")

okie


In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
def loadpdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [4]:
extracted_data = loadpdf("data/")

In [5]:
#extracted_data

In [6]:
# create text chunks 
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print("Length of my chunk:", len(text_chunks))

Length of my chunk: 39994


In [8]:
# text_chunks

In [9]:
from langchain_huggingface import HuggingFaceEmbeddings

def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_huggingface_embeddings()

c:\Users\Ranjeet Kumar\anaconda3\envs\mchatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

load_dotenv()

PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
PINECONE_API_ENV = os.environ.get("PINECONE_API_ENV")

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medical-cbot"

In [12]:
# Create index if not exists
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

In [13]:
# Now use LangChain PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name,
)

In [14]:
# docsearch

In [15]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":5})

In [22]:
retrieved_docs = retriever.invoke("ion")

In [23]:
retrieved_docs

[Document(id='9dfe8494-c29c-45d2-bb8d-ef7dc5f5c7c8', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 1306.0, 'page_label': '1277', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'data\\Medical book.pdf', 'total_pages': 4505.0}, page_content='tively charged. Positively charged ions are called\ncations, and are formed when an atom or atom\ngroup loses electrons. The most important cations in\nthe human body are sodium, potassium, magnesium,\nand calcium ions. Negatively charged ions are called\nanions, and are formed when an atom or atom group\ngains electrons. The principal anions in the body\ninclude bicarbonate, chloride, phosphate, and sulfate\nions, as well as ions formed by certain protein\ncompounds or organic acids.'),
 Document(id='41575e47-466a-43b2-88fc-fbe5b1a02012', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:

In [56]:
from langchain_community.llms import Together
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Initialize Together API LLM
TOGETHER_API_KEY = os.environ.get("TOGETHER_API_KEY")

llm = Together(
    model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo",
    temperature=0.4,
    max_tokens=500,
    together_api_key=TOGETHER_API_KEY
)

system_prompt = (
    "Answer the question using only the given context. "
    "Return the answer in concise bullet points. "
    "Do not explain or add reasoning. "
    "Return only the final answer, in one or two sentences. "
    "If the answer is not in the context, respond with 'I don't know.'\n\n"
    "Context:\n{context}"
)

# Step 2: Prepare the chat prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Step 3: Create the retrieval pipeline
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Combine retriever and LLM QA chain
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

try: 
    question = "what is Warts and its Description, Causes, symptoms, Diagnosis, Treatment and Prevention"
    response = rag_chain.invoke({"input": question})
    print(f"Question: {question}")
    print(f"Answer: {response['answer']}")

except Exception as e:
    print(f"\nError: {e}")
    print("Check Pinecone connection and API keys.")

Question: what is Warts and its Description, Causes, symptoms, Diagnosis, Treatment and Prevention
Answer: ?

Warts: 
Description: Warts are small, benign growths caused by a viral infection of the skin or mucous membrane. 
Causes: Warts are caused by the human papilloma virus (HPV). 
Symptoms: The symptoms include bleeding, pain, and odor as well as the visible warts. 
Diagnosis: The diagnosis is usually made by examining scrapings from the warts under a darkfield microscope. 
Treatment: No treatment for genital warts is completely effective because therapy depends on destroying skin infected with the virus. 
Prevention: I don't know.
